SCENARIO 1

In [0]:

#The task csv files have been saved to Azure datalake 

mount_point = '/mnt/data'
source = dbutils.secrets.get(scope='bjohnsen-secrets', key='data')

# Check if mount point is already mounted
if mount_point in [mount.mountPoint for mount in dbutils.fs.mounts()]:
    dbutils.fs.refreshMounts()
else:
    # Mount the Azure Data Lake Gen2 data container
    configs = {
        "fs.azure.account.auth.type": "OAuth",
        "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
        "fs.azure.account.oauth2.client.endpoint": dbutils.secrets.get(scope="bjohnsen-secrets", key="client-url"),
        "fs.azure.account.oauth2.client.id": dbutils.secrets.get(scope="bjohnsen-secrets", key="clientid"),
        "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="bjohnsen-secrets", key="clientsecret"),
        "fs.azure.account.oauth2.client.endpoint.staging": dbutils.secrets.get(scope="bjohnsen-secrets", key="data")
    }
    dbutils.fs.mount(source=source, mount_point=mount_point, extra_configs=configs)

# List all files in the staging folder
file_list = dbutils.fs.ls("/mnt/data/staging")

# Print the names of all files
for file in file_list:
    print(file.name)

Mounts successfully refreshed.
ownership_og_data.csv
ownership_updated_data.csv


In [0]:
# Read ownership_og_data.csv file into a dataframe
df = spark.read.format("csv") \
            .option("header", "true") \
            .option("delimiter", ",") \
            .option("inferSchema", "true") \
            .load("/mnt/data/staging/ownership_og_data.csv")

In [0]:
# optionally repartition the dataframe
df_repartitioned = df.repartition(2)

In [0]:
#display the dataframe
display(df_repartitioned)

Title Number,Tenure,Property Address,District,County,Region,Postcode,Multiple Address Indicator,Price Paid,Proprietor Name (1),Company Registration No. (1),Proprietorship Category (1),Proprietor (1) Address (1),Proprietor (1) Address (2),Proprietor (1) Address (3),Proprietor Name (2),Company Registration No. (2)
CH408942,Freehold,"the site of a gas governor site on the south side of Littlecote Gardens, Appleton",WARRINGTON,WARRINGTON,NORTH WEST,null,N,null,CADENT GAS LIMITED,10080864,Limited Company or Public Limited Company,"Ashbrook Court, Prologis Park, Central Boulevard, Keresley End, Coventry CV7 8PE",null,null,null,null
BK439278,Leasehold,"Apartment 9, Charters Garden House, Charters Road, Ascot and two parking spaces numbered B9 (SL5 9GE)",WINDSOR AND MAIDENHEAD,WINDSOR AND MAIDENHEAD,SOUTH EAST,SL5 9GE,N,null,ARORA HEATHROW T5 LIMITED,5202590,Limited Company or Public Limited Company,"2 World Business Centre Heathrow, Newall Road, London Heathrow Airport, Hounslow TW6 2SF",null,null,null,null
BK311066,Freehold,"37 Perryman Way, Slough (SL2 2HF)",SLOUGH,SLOUGH,SOUTH EAST,SL2 2HF,N,null,POONAM INVESTMENTS LIMITED,12777394,Limited Company or Public Limited Company,"112 Morden Road, London SW19 3BP",null,null,null,null
BM134958,Freehold,"553 South Row, Milton Keynes (MK9 3DQ)",MILTON KEYNES,MILTON KEYNES,SOUTH EAST,MK9 3DQ,N,null,CRESCENT ARCH PROPERTIES 2 LTD,11841936,Limited Company or Public Limited Company,"70-72 Victoria Road, Ruislip HA4 0AH",null,null,null,null
WM16094,Freehold,"55 Raddlebarn Road, Selly Oak, Birmingham (B29 6HQ)",BIRMINGHAM,WEST MIDLANDS,WEST MIDLANDS,B29 6HQ,N,180000,AXA PORTFOLIO SERVICES LIMITED,1128611,Limited Company or Public Limited Company,"5 Old Broad Street, London EC2N 1AD","141 Castle Street, Salisbury SP1 3TB",null,null,null
CH168259,Freehold,"124 Vicars Cross Road, Great Boughton (CH3 5NP)",CHESHIRE WEST AND CHESTER,CHESHIRE WEST AND CHESTER,NORTH WEST,CH3 5NP,N,285000,FRESH SPACE LIVING LTD,12668624,Limited Company or Public Limited Company,"7 Telford Court, Chestergates, Chester CH1 6LT","15 The Plantation, Blackheath, London SE3 0AB",null,null,null
BM137686,Leasehold,"12 Bourbon Street, Aylesbury (HP20 2RR)",BUCKINGHAMSHIRE,BUCKINGHAMSHIRE,SOUTH EAST,HP20 2RR,N,null,HARRIS HOMES RENTS AND LETTINGS 2 LTD,13331867,Limited Company or Public Limited Company,"12 Bourbon Street, Aylesbury HP20 2RR",null,null,null,null
CH395408,Leasehold,"the site of a Gas Governor on the east side of Whitley Avenue, Warrington",WARRINGTON,WARRINGTON,NORTH WEST,null,N,null,CADENT GAS LIMITED,10080864,Limited Company or Public Limited Company,"Ashbrook Court, Prologis Park, Central Boulevard, Keresley End, Coventry CV7 8PE",null,null,null,null
CB453472,Leasehold,"land at Fieldfare Drive, Stanground, Peterborough (PE2 8SR) Site reference - NG_PP_001378",CITY OF PETERBOROUGH,CITY OF PETERBOROUGH,EAST ANGLIA,PE2 8SR,N,null,CADENT GAS LIMITED,10080864,Limited Company or Public Limited Company,"Ashbrook Court, Prologis Park, Central Boulevard, Keresley End, Coventry CV7 8PE",null,null,null,null
BK461382,Leasehold,"Unit 3A, Slough Retail Park, Twinches Lane, Slough (SL1 5AL)",SLOUGH,SLOUGH,SOUTH EAST,SL1 5AL,N,null,STEINHOFF UK GROUP PROPERTIES LIMITED,1024575,Limited Company or Public Limited Company,"Festival House, Jessop Avenue, Cheltenham GL50 3SH","520 Europa Boulevard, Westbrook, Warrington WA5 7TP",null,null,null


In [0]:
# Perform some transformation

from pyspark.sql.functions import col

# Renaming and making field names valid
df_og_cleaned = df_repartitioned.withColumnRenamed("Title Number", "Title_Number") \
    .withColumnRenamed("Tenure", "Tenure") \
    .withColumnRenamed("Property Address", "Property_Address") \
    .withColumnRenamed("District", "District") \
    .withColumnRenamed("County", "County") \
    .withColumnRenamed("Region", "Region") \
    .withColumnRenamed("Postcode", "Postcode") \
    .withColumnRenamed("Multiple Address Indicator", "Multiple_Address_Indicator") \
    .withColumnRenamed("Price Paid", "Price_Paid") \
    .withColumnRenamed("Proprietor Name (1)", "Proprietor_Name_1") \
    .withColumnRenamed("Company Registration No. (1)", "Company_Registration_No_1") \
    .withColumnRenamed("Proprietorship Category (1)", "Proprietorship_Category_1") \
    .withColumnRenamed("Proprietor (1) Address (1)", "Proprietor_1_Address_1") \
    .withColumnRenamed("Proprietor (1) Address (2)", "Proprietor_1_Address_2") \
    .withColumnRenamed("Proprietor (1) Address (3)", "Proprietor_1_Address_3") \
    .withColumnRenamed("Proprietor Name (2)", "Proprietor_Name_2") \
    .withColumnRenamed("Company Registration No. (2)", "Company_Registration_No_2")

display(df_og_cleaned)


Title_Number,Tenure,Property_Address,District,County,Region,Postcode,Multiple_Address_Indicator,Price_Paid,Proprietor_Name_1,Company_Registration_No_1,Proprietorship_Category_1,Proprietor_1_Address_1,Proprietor_1_Address_2,Proprietor_1_Address_3,Proprietor_Name_2,Company_Registration_No_2
CH408942,Freehold,"the site of a gas governor site on the south side of Littlecote Gardens, Appleton",WARRINGTON,WARRINGTON,NORTH WEST,null,N,null,CADENT GAS LIMITED,10080864,Limited Company or Public Limited Company,"Ashbrook Court, Prologis Park, Central Boulevard, Keresley End, Coventry CV7 8PE",null,null,null,null
BK439278,Leasehold,"Apartment 9, Charters Garden House, Charters Road, Ascot and two parking spaces numbered B9 (SL5 9GE)",WINDSOR AND MAIDENHEAD,WINDSOR AND MAIDENHEAD,SOUTH EAST,SL5 9GE,N,null,ARORA HEATHROW T5 LIMITED,5202590,Limited Company or Public Limited Company,"2 World Business Centre Heathrow, Newall Road, London Heathrow Airport, Hounslow TW6 2SF",null,null,null,null
BK311066,Freehold,"37 Perryman Way, Slough (SL2 2HF)",SLOUGH,SLOUGH,SOUTH EAST,SL2 2HF,N,null,POONAM INVESTMENTS LIMITED,12777394,Limited Company or Public Limited Company,"112 Morden Road, London SW19 3BP",null,null,null,null
BM134958,Freehold,"553 South Row, Milton Keynes (MK9 3DQ)",MILTON KEYNES,MILTON KEYNES,SOUTH EAST,MK9 3DQ,N,null,CRESCENT ARCH PROPERTIES 2 LTD,11841936,Limited Company or Public Limited Company,"70-72 Victoria Road, Ruislip HA4 0AH",null,null,null,null
WM16094,Freehold,"55 Raddlebarn Road, Selly Oak, Birmingham (B29 6HQ)",BIRMINGHAM,WEST MIDLANDS,WEST MIDLANDS,B29 6HQ,N,180000,AXA PORTFOLIO SERVICES LIMITED,1128611,Limited Company or Public Limited Company,"5 Old Broad Street, London EC2N 1AD","141 Castle Street, Salisbury SP1 3TB",null,null,null
CH168259,Freehold,"124 Vicars Cross Road, Great Boughton (CH3 5NP)",CHESHIRE WEST AND CHESTER,CHESHIRE WEST AND CHESTER,NORTH WEST,CH3 5NP,N,285000,FRESH SPACE LIVING LTD,12668624,Limited Company or Public Limited Company,"7 Telford Court, Chestergates, Chester CH1 6LT","15 The Plantation, Blackheath, London SE3 0AB",null,null,null
BM137686,Leasehold,"12 Bourbon Street, Aylesbury (HP20 2RR)",BUCKINGHAMSHIRE,BUCKINGHAMSHIRE,SOUTH EAST,HP20 2RR,N,null,HARRIS HOMES RENTS AND LETTINGS 2 LTD,13331867,Limited Company or Public Limited Company,"12 Bourbon Street, Aylesbury HP20 2RR",null,null,null,null
CH395408,Leasehold,"the site of a Gas Governor on the east side of Whitley Avenue, Warrington",WARRINGTON,WARRINGTON,NORTH WEST,null,N,null,CADENT GAS LIMITED,10080864,Limited Company or Public Limited Company,"Ashbrook Court, Prologis Park, Central Boulevard, Keresley End, Coventry CV7 8PE",null,null,null,null
CB453472,Leasehold,"land at Fieldfare Drive, Stanground, Peterborough (PE2 8SR) Site reference - NG_PP_001378",CITY OF PETERBOROUGH,CITY OF PETERBOROUGH,EAST ANGLIA,PE2 8SR,N,null,CADENT GAS LIMITED,10080864,Limited Company or Public Limited Company,"Ashbrook Court, Prologis Park, Central Boulevard, Keresley End, Coventry CV7 8PE",null,null,null,null
BK461382,Leasehold,"Unit 3A, Slough Retail Park, Twinches Lane, Slough (SL1 5AL)",SLOUGH,SLOUGH,SOUTH EAST,SL1 5AL,N,null,STEINHOFF UK GROUP PROPERTIES LIMITED,1024575,Limited Company or Public Limited Company,"Festival House, Jessop Avenue, Cheltenham GL50 3SH","520 Europa Boulevard, Westbrook, Warrington WA5 7TP",null,null,null


In [0]:
#save in the ADLS container in delta format

df_og_cleaned.write.format("delta").mode("overwrite").save("/mnt/data/transformed")


SCENARIO 2

In [0]:
# Read ownership_updated_data.csv file into a dataframe
df_updated = spark.read.format("csv") \
            .option("header", "true") \
            .option("delimiter", ",") \
            .option("inferSchema", "true") \
            .load("/mnt/data/staging/ownership_updated_data.csv")

display (df_updated)

Title Number,Tenure,Property Address,District,County,Region,Postcode,Multiple Address Indicator,Price Paid,Proprietor Name (1),Company Registration No. (1),Proprietorship Category (1),Proprietor (1) Address (1),Proprietor (1) Address (2),Proprietor (1) Address (3),Proprietor Name (2),Company Registration No. (2)
WA44352,Leasehold,"94 Rhondda Street, Swansea (SA1 6ET)",SWANSEA,SWANSEA,WALES,null,N,140000,AG PROPERTIES & MAINTENANCE LTD,10785500,Local Authority,"Unit 5, Llewellyns Quay, Port Talbot SA13 1RF",null,null,null,null
WM16094,Leasehold,"55 Raddlebarn Road, Selly Oak, Birmingham (B29 6HQ)",BIRMINGHAM,WEST MIDLANDS,WEST MIDLANDS,B29 6HQ,N,180000,AXA PORTFOLIO SERVICES LIMITED,1128611,Limited Company or Public Limited Company,null,"141 Castle Street, Salisbury SP1 3TB",null,null,null
WM59658,Freehold,"1-40 Old Rectory Gardens, Aldridge, (WS9 8UP)",WINDSOR AND MAIDENHEAD,WEST MIDLANDS,WEST MIDLANDS,WS9 8UP,N,null,ACCORD HOUSING ASSOCIATION LIMITED,27052R,Local Authority,"178 Birmingham Road, West Bromwich B70 6QG",null,null,null,null
WM98474,Freehold,"Flats 1 to 12 (inc) and 14 to 52 (inc) Maryvale, Glebe Street, Walsall (WS1 3LP)",WALSALL,WEST MIDLANDS,WEST MIDLANDS,WS1 3LP,N,800,ACCORD HOUSING ASSOCIATION LIMITED,27052R,Local Authority,"178 Birmingham Road, West Bromwich B70 6QG",null,null,null,null
BD153591,Freehold,"land and buildings on the North side of Harrold Priory, Goldington, Bedford",BEDFORD,BEDFORD,SOUTH EAST,null,N,null,CADENT GAS LIMITED,10080864,Limited Company or Public Limited Company,"Ashbrook Court, Prologis Park, Central Boulevard, Keresley End, Coventry CV7 8PE",null,null,null,null
BM453325,Leasehold,"2-20 (Even), Bengrove, Wolverton Mill, Milton Keynes ( MK12 5SG)",MILTON KEYNES,MILTON KEYNES,SOUTH EAST,MK12 5SG,N,null,GRAND UNION HOUSING GROUP LIMITED,7853,Community Benefit Society (Company),"K2, Timbold Drive, Kents Hill, Milton Keynes MK7 6BZ",null,null,null,null
CH371644,Leasehold,"Gas Governor, Gadbrook Business Centre, Rudheath, Northwich",CHESHIRE WEST AND CHESTER,CHESHIRE WEST AND CHESTER,NORTH WEST,null,N,900,CADENT GAS LIMITED,10080864,Limited Company or Public Limited Company,"Ashbrook Court, Prologis Park, Central Boulevard, Keresley End, Coventry CV7 8PE",null,null,null,null
BK311066,Freehold,"37 Perryman Way, Slough (SL2 2HF)",SLOUGH,SLOUGH,SOUTH EAST,SL2 2HF,N,null,POONAM INVESTMENTS LIMITED,12777394,Limited Company or Public Limited Company,"112 Morden Road, London SW19 3BP",null,null,null,null
BK512819,Leasehold,"Flat 2, 5 Newark Street, Reading (RG1 2SR)",READING,READING,SOUTH EAST,RG1 2SR,N,null,SAGE HOUSING LIMITED,5489096,Housing Association/Society (Company),"5th Floor, Orion House, 5 Upper St. Martin's Lane, London WC2H 9EA",null,null,null,null
BL117675,Freehold,"74 Brooklyn Road, Bristol (BS13 7JZ)",CITY OF BRISTOL,CITY OF BRISTOL,SOUTH WEST,BS13 7JZ,N,2300000,THE CITY COUNCIL OF BRISTOL,null,Local Authority,"City Hall, College Green, Bristol BS1 5TR",null,null,null,null


In [0]:
# Adjust the column name to match the ownership_og_data
# Renaming and making field names valid
df_updated_cleaned = df_updated.withColumnRenamed("Title Number", "Title_Number") \
    .withColumnRenamed("Tenure", "Tenure") \
    .withColumnRenamed("Property Address", "Property_Address") \
    .withColumnRenamed("District", "District") \
    .withColumnRenamed("County", "County") \
    .withColumnRenamed("Region", "Region") \
    .withColumnRenamed("Postcode", "Postcode") \
    .withColumnRenamed("Multiple Address Indicator", "Multiple_Address_Indicator") \
    .withColumnRenamed("Price Paid", "Price_Paid") \
    .withColumnRenamed("Proprietor Name (1)", "Proprietor_Name_1") \
    .withColumnRenamed("Company Registration No. (1)", "Company_Registration_No_1") \
    .withColumnRenamed("Proprietorship Category (1)", "Proprietorship_Category_1") \
    .withColumnRenamed("Proprietor (1) Address (1)", "Proprietor_1_Address_1") \
    .withColumnRenamed("Proprietor (1) Address (2)", "Proprietor_1_Address_2") \
    .withColumnRenamed("Proprietor (1) Address (3)", "Proprietor_1_Address_3") \
    .withColumnRenamed("Proprietor Name (2)", "Proprietor_Name_2") \
    .withColumnRenamed("Company Registration No. (2)", "Company_Registration_No_2")

display(df_updated_cleaned)

Title_Number,Tenure,Property_Address,District,County,Region,Postcode,Multiple_Address_Indicator,Price_Paid,Proprietor_Name_1,Company_Registration_No_1,Proprietorship_Category_1,Proprietor_1_Address_1,Proprietor_1_Address_2,Proprietor_1_Address_3,Proprietor_Name_2,Company_Registration_No_2
WA44352,Leasehold,"94 Rhondda Street, Swansea (SA1 6ET)",SWANSEA,SWANSEA,WALES,null,N,140000,AG PROPERTIES & MAINTENANCE LTD,10785500,Local Authority,"Unit 5, Llewellyns Quay, Port Talbot SA13 1RF",null,null,null,null
WM16094,Leasehold,"55 Raddlebarn Road, Selly Oak, Birmingham (B29 6HQ)",BIRMINGHAM,WEST MIDLANDS,WEST MIDLANDS,B29 6HQ,N,180000,AXA PORTFOLIO SERVICES LIMITED,1128611,Limited Company or Public Limited Company,null,"141 Castle Street, Salisbury SP1 3TB",null,null,null
WM59658,Freehold,"1-40 Old Rectory Gardens, Aldridge, (WS9 8UP)",WINDSOR AND MAIDENHEAD,WEST MIDLANDS,WEST MIDLANDS,WS9 8UP,N,null,ACCORD HOUSING ASSOCIATION LIMITED,27052R,Local Authority,"178 Birmingham Road, West Bromwich B70 6QG",null,null,null,null
WM98474,Freehold,"Flats 1 to 12 (inc) and 14 to 52 (inc) Maryvale, Glebe Street, Walsall (WS1 3LP)",WALSALL,WEST MIDLANDS,WEST MIDLANDS,WS1 3LP,N,800,ACCORD HOUSING ASSOCIATION LIMITED,27052R,Local Authority,"178 Birmingham Road, West Bromwich B70 6QG",null,null,null,null
BD153591,Freehold,"land and buildings on the North side of Harrold Priory, Goldington, Bedford",BEDFORD,BEDFORD,SOUTH EAST,null,N,null,CADENT GAS LIMITED,10080864,Limited Company or Public Limited Company,"Ashbrook Court, Prologis Park, Central Boulevard, Keresley End, Coventry CV7 8PE",null,null,null,null
BM453325,Leasehold,"2-20 (Even), Bengrove, Wolverton Mill, Milton Keynes ( MK12 5SG)",MILTON KEYNES,MILTON KEYNES,SOUTH EAST,MK12 5SG,N,null,GRAND UNION HOUSING GROUP LIMITED,7853,Community Benefit Society (Company),"K2, Timbold Drive, Kents Hill, Milton Keynes MK7 6BZ",null,null,null,null
CH371644,Leasehold,"Gas Governor, Gadbrook Business Centre, Rudheath, Northwich",CHESHIRE WEST AND CHESTER,CHESHIRE WEST AND CHESTER,NORTH WEST,null,N,900,CADENT GAS LIMITED,10080864,Limited Company or Public Limited Company,"Ashbrook Court, Prologis Park, Central Boulevard, Keresley End, Coventry CV7 8PE",null,null,null,null
BK311066,Freehold,"37 Perryman Way, Slough (SL2 2HF)",SLOUGH,SLOUGH,SOUTH EAST,SL2 2HF,N,null,POONAM INVESTMENTS LIMITED,12777394,Limited Company or Public Limited Company,"112 Morden Road, London SW19 3BP",null,null,null,null
BK512819,Leasehold,"Flat 2, 5 Newark Street, Reading (RG1 2SR)",READING,READING,SOUTH EAST,RG1 2SR,N,null,SAGE HOUSING LIMITED,5489096,Housing Association/Society (Company),"5th Floor, Orion House, 5 Upper St. Martin's Lane, London WC2H 9EA",null,null,null,null
BL117675,Freehold,"74 Brooklyn Road, Bristol (BS13 7JZ)",CITY OF BRISTOL,CITY OF BRISTOL,SOUTH WEST,BS13 7JZ,N,2300000,THE CITY COUNCIL OF BRISTOL,null,Local Authority,"City Hall, College Green, Bristol BS1 5TR",null,null,null,null


In [0]:
#save in the ADLS container in delta format

df_updated_cleaned.write.format("delta").mode("overwrite").save("/mnt/data/transformed")

CHANGE ONLY UPDATE

In [0]:
# Step 1: Identify the records that have been updated
updated_records = df_updated_cleaned.join(df_og_cleaned, on="Title_Number", how="inner")

# Step 2: Select the updated records from df_updated_cleaned
# Provide column names explicitly to resolve ambiguity
updated_columns = [df_updated_cleaned[col] for col in df_updated_cleaned.columns]
updated_records = updated_records.select(updated_columns)

# Show the updated_records
# updated_records.show()

# Step 3: Join the selected updated records with df_og_cleaned on Title_Number
joined_df = df_og_cleaned.join(updated_records, on="Title_Number", how="left_anti")

# Step 4: Union updated df_og_cleaned with unchanged records from df_updated_cleaned
df_result = joined_df.union(df_updated_cleaned)

# Show the result
display(df_result)


Title_Number,Tenure,Property_Address,District,County,Region,Postcode,Multiple_Address_Indicator,Price_Paid,Proprietor_Name_1,Company_Registration_No_1,Proprietorship_Category_1,Proprietor_1_Address_1,Proprietor_1_Address_2,Proprietor_1_Address_3,Proprietor_Name_2,Company_Registration_No_2
WA44352,Leasehold,"94 Rhondda Street, Swansea (SA1 6ET)",SWANSEA,SWANSEA,WALES,null,N,140000,AG PROPERTIES & MAINTENANCE LTD,10785500,Local Authority,"Unit 5, Llewellyns Quay, Port Talbot SA13 1RF",null,null,null,null
WM16094,Leasehold,"55 Raddlebarn Road, Selly Oak, Birmingham (B29 6HQ)",BIRMINGHAM,WEST MIDLANDS,WEST MIDLANDS,B29 6HQ,N,180000,AXA PORTFOLIO SERVICES LIMITED,1128611,Limited Company or Public Limited Company,null,"141 Castle Street, Salisbury SP1 3TB",null,null,null
WM59658,Freehold,"1-40 Old Rectory Gardens, Aldridge, (WS9 8UP)",WINDSOR AND MAIDENHEAD,WEST MIDLANDS,WEST MIDLANDS,WS9 8UP,N,null,ACCORD HOUSING ASSOCIATION LIMITED,27052R,Local Authority,"178 Birmingham Road, West Bromwich B70 6QG",null,null,null,null
WM98474,Freehold,"Flats 1 to 12 (inc) and 14 to 52 (inc) Maryvale, Glebe Street, Walsall (WS1 3LP)",WALSALL,WEST MIDLANDS,WEST MIDLANDS,WS1 3LP,N,800,ACCORD HOUSING ASSOCIATION LIMITED,27052R,Local Authority,"178 Birmingham Road, West Bromwich B70 6QG",null,null,null,null
BD153591,Freehold,"land and buildings on the North side of Harrold Priory, Goldington, Bedford",BEDFORD,BEDFORD,SOUTH EAST,null,N,null,CADENT GAS LIMITED,10080864,Limited Company or Public Limited Company,"Ashbrook Court, Prologis Park, Central Boulevard, Keresley End, Coventry CV7 8PE",null,null,null,null
BM453325,Leasehold,"2-20 (Even), Bengrove, Wolverton Mill, Milton Keynes ( MK12 5SG)",MILTON KEYNES,MILTON KEYNES,SOUTH EAST,MK12 5SG,N,null,GRAND UNION HOUSING GROUP LIMITED,7853,Community Benefit Society (Company),"K2, Timbold Drive, Kents Hill, Milton Keynes MK7 6BZ",null,null,null,null
CH371644,Leasehold,"Gas Governor, Gadbrook Business Centre, Rudheath, Northwich",CHESHIRE WEST AND CHESTER,CHESHIRE WEST AND CHESTER,NORTH WEST,null,N,900,CADENT GAS LIMITED,10080864,Limited Company or Public Limited Company,"Ashbrook Court, Prologis Park, Central Boulevard, Keresley End, Coventry CV7 8PE",null,null,null,null
BK311066,Freehold,"37 Perryman Way, Slough (SL2 2HF)",SLOUGH,SLOUGH,SOUTH EAST,SL2 2HF,N,null,POONAM INVESTMENTS LIMITED,12777394,Limited Company or Public Limited Company,"112 Morden Road, London SW19 3BP",null,null,null,null
BK512819,Leasehold,"Flat 2, 5 Newark Street, Reading (RG1 2SR)",READING,READING,SOUTH EAST,RG1 2SR,N,null,SAGE HOUSING LIMITED,5489096,Housing Association/Society (Company),"5th Floor, Orion House, 5 Upper St. Martin's Lane, London WC2H 9EA",null,null,null,null
BL117675,Freehold,"74 Brooklyn Road, Bristol (BS13 7JZ)",CITY OF BRISTOL,CITY OF BRISTOL,SOUTH WEST,BS13 7JZ,N,2300000,THE CITY COUNCIL OF BRISTOL,null,Local Authority,"City Hall, College Green, Bristol BS1 5TR",null,null,null,null


In [0]:
# Write the result into the result directory
df_result.write.format("delta").mode("overwrite").save("/mnt/data/result")